In [31]:
import pandas as pd
from prophet import Prophet
import plotly.express as px
from datetime import datetime, timedelta
import yahoo_fin.stock_info as si

# Function to fetch historical data for a stock based on today's date
def fetch_historical_data(ticker, start_date):
    try:
        end_date = datetime.today().strftime('%Y-%m-%d')
        data = si.get_data(ticker, start_date=start_date, end_date=end_date)
        data = data.reset_index()
        data = data[['index', 'close']]  # Ensure only needed columns are included
        data.rename(columns={'index': 'Date', 'close': 'Close'}, inplace=True)

        # Compute Close Shift
        data['Close Shift'] = data['Close'].diff()

        return data

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def display_forecasts_from_data(df, ticker, periods=30):
    if df.empty:
        print("Error: Empty DataFrame")
        return

    # Rename columns to match Prophet's expected format and use 'Close Shift' as target
    df.rename(columns={'Date': 'ds', 'Close Shift': 'y'}, inplace=True)

    # Drop any rows with NaN values (which will be the first row after diff())
    df.dropna(inplace=True)

    model = Prophet()
    model.fit(df)

    # Generate future DataFrame starting from the last date +1
    last_date = df['ds'].iloc[-1]
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=periods, freq='D')
    future = pd.DataFrame({'ds': future_dates})

    forecast = model.predict(future)

    # Add forecasted close shifts to the last known close price to predict actual prices
    last_known_close = df['Close'].iloc[-1]  # Use 'Close' from the original DataFrame before renaming
    forecast['Forecasted Close'] = last_known_close + forecast['yhat'].cumsum()

    # Print and plot the results
    print("Forecasted Close Prices:")
    print(forecast[['ds', 'Forecasted Close']])

    fig1 = px.line(forecast, x='ds', y='Forecasted Close', labels={'Forecasted Close': 'Forecast'}, title=f'{ticker} Forecast')
    fig2 = px.line(df, x='ds', y='Close', labels={'Close': 'Historical Data'}, title=f'{ticker} Historical Data')

    fig1.show()
    fig2.show()

# Example usage
ticker = 'TSLA'
start_date = datetime.today() - timedelta(days=3650)  # 10 years ago

try:
    historical_data = fetch_historical_data(ticker, start_date)
    if historical_data is not None:
        display_forecasts_from_data(historical_data, ticker)
except Exception as e:
    print(f"Error processing forecasts: {e}")

00:03:50 - cmdstanpy - INFO - Chain [1] start processing
00:03:50 - cmdstanpy - INFO - Chain [1] done processing


Forecasted Close Prices:
           ds  Forecasted Close
0  2024-04-20        146.163288
1  2024-04-21        145.252188
2  2024-04-22        144.952653
3  2024-04-23        144.046165
4  2024-04-24        143.058586
5  2024-04-25        141.646622
6  2024-04-26        140.394801
7  2024-04-27        139.301873
8  2024-04-28        138.176141
9  2024-04-29        137.656667
10 2024-04-30        136.529173
11 2024-05-01        135.324771
12 2024-05-02        133.706257
13 2024-05-03        132.264896
14 2024-05-04        131.006596
15 2024-05-05        129.747003
16 2024-05-06        129.132444
17 2024-05-07        127.955587
18 2024-05-08        126.753904
19 2024-05-09        125.195743
20 2024-05-10        123.876881
21 2024-05-11        122.806528
22 2024-05-12        121.802249
23 2024-05-13        121.510807
24 2024-05-14        120.723738
25 2024-05-15        119.975800
26 2024-05-16        118.931052
27 2024-05-17        118.179486
28 2024-05-18        117.723133
29 2024-05-19  

In [1]:
import pandas as pd
from prophet import Prophet
import plotly.express as px
from datetime import datetime, timedelta
import yahoo_fin.stock_info as si

# Function to fetch historical data for a stock based on today's date
def fetch_historical_data(ticker, start_date):
    try:
        end_date = datetime.today().strftime('%Y-%m-%d')
        data = si.get_data(ticker, start_date=start_date, end_date=end_date)
        data = data.reset_index()
        data = data[['index', 'close']]  # Ensure only needed columns are included
        data.rename(columns={'index': 'Date', 'close': 'Close'}, inplace=True)
        data['Close Shift'] = data['Close'].diff()
        return data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

def display_forecasts_from_data(df, ticker, periods=30):
    if df.empty:
        print(f"Error: Empty DataFrame for {ticker}")
        return

    df.rename(columns={'Date': 'ds', 'Close Shift': 'y'}, inplace=True)
    df.dropna(inplace=True)

    model = Prophet()
    model.fit(df)

    last_date = df['ds'].iloc[-1]
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=periods, freq='D')
    future = pd.DataFrame({'ds': future_dates})

    forecast = model.predict(future)
    last_known_close = df['Close'].iloc[-1]  # Ensure this column exists before renaming
    forecast['Forecasted Close'] = last_known_close + forecast['yhat'].cumsum()

    print(f"Forecasted Close Prices for {ticker}:")
    print(forecast[['ds', 'Forecasted Close']])

    fig = px.line(forecast, x='ds', y='Forecasted Close', labels={'Forecasted Close': 'Forecast'}, title=f'{ticker} Forecast')
    fig.show()

def process_multiple_tickers(tickers, start_date):
    for ticker in tickers:
        historical_data = fetch_historical_data(ticker, start_date)
        if not historical_data.empty:
            display_forecasts_from_data(historical_data, ticker)

# Example usage
tickers = ['AAPL', 'GOOGL', 'MSFT']  # Array of tickers
start_date = datetime.today() - timedelta(days=3650)  # 10 years ago

process_multiple_tickers(tickers, start_date)


c:\Users\kahbe\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
15:34:20 - cmdstanpy - INFO - Chain [1] start processing
15:34:21 - cmdstanpy - INFO - Chain [1] done processing


Forecasted Close Prices for AAPL:
           ds  Forecasted Close
0  2024-04-23        165.759312
1  2024-04-24        165.689177
2  2024-04-25        165.505465
3  2024-04-26        165.357998
4  2024-04-27        165.219861
5  2024-04-28        165.078524
6  2024-04-29        165.067000
7  2024-04-30        164.957410
8  2024-05-01        164.864980
9  2024-05-02        164.665639
10 2024-05-03        164.509230
11 2024-05-04        164.368832
12 2024-05-05        164.231868
13 2024-05-06        164.231271
14 2024-05-07        164.139037
15 2024-05-08        164.070226
16 2024-05-09        163.900548
17 2024-05-10        163.779578
18 2024-05-11        163.680074
19 2024-05-12        163.589084
20 2024-05-13        163.639110
21 2024-05-14        163.601662
22 2024-05-15        163.591262
23 2024-05-16        163.483035
24 2024-05-17        163.425926
25 2024-05-18        163.392027
26 2024-05-19        163.367688
27 2024-05-20        163.484697
28 2024-05-21        163.513838
29 202

15:34:23 - cmdstanpy - INFO - Chain [1] start processing
15:34:24 - cmdstanpy - INFO - Chain [1] done processing


Forecasted Close Prices for GOOGL:
           ds  Forecasted Close
0  2024-04-23        156.251247
1  2024-04-24        156.305662
2  2024-04-25        156.330232
3  2024-04-26        156.281985
4  2024-04-27        156.669378
5  2024-04-28        157.065497
6  2024-04-29        157.047083
7  2024-04-30        157.065439
8  2024-05-01        157.178983
9  2024-05-02        157.273756
10 2024-05-03        157.305616
11 2024-05-04        157.781643
12 2024-05-05        158.273373
13 2024-05-06        158.355857
14 2024-05-07        158.478600
15 2024-05-08        158.698156
16 2024-05-09        158.898680
17 2024-05-10        159.034153
18 2024-05-11        159.609840
19 2024-05-12        160.195553
20 2024-05-13        160.364751
21 2024-05-14        160.565512
22 2024-05-15        160.853153
23 2024-05-16        161.110807
24 2024-05-17        161.291666
25 2024-05-18        161.900448
26 2024-05-19        162.506668
27 2024-05-20        162.683736
28 2024-05-21        162.879917
29 20

15:34:25 - cmdstanpy - INFO - Chain [1] start processing
15:34:25 - cmdstanpy - INFO - Chain [1] done processing


Forecasted Close Prices for MSFT:
           ds  Forecasted Close
0  2024-04-23        400.952093
1  2024-04-24        401.230317
2  2024-04-25        401.436403
3  2024-04-26        401.596668
4  2024-04-27        401.527615
5  2024-04-28        401.457388
6  2024-04-29        401.495655
7  2024-04-30        401.474408
8  2024-05-01        401.745745
9  2024-05-02        401.952025
10 2024-05-03        402.120179
11 2024-05-04        402.067283
12 2024-05-05        402.021982
13 2024-05-06        402.094348
14 2024-05-07        402.116650
15 2024-05-08        402.441114
16 2024-05-09        402.710060
17 2024-05-10        402.950199
18 2024-05-11        402.978198
19 2024-05-12        403.022099
20 2024-05-13        403.191184
21 2024-05-14        403.316748
22 2024-05-15        403.749881
23 2024-05-16        404.131615
24 2024-05-17        404.487254
25 2024-05-18        404.631961
26 2024-05-19        404.792210
27 2024-05-20        405.075689
28 2024-05-21        405.312103
29 202